In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
import ogb
from tqdm import tqdm
import hiplot as hip
from copy import deepcopy
import datetime

In [3]:
import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch.utils.data import Subset, TensorDataset

In [4]:
cwd = os.getcwd()
print(cwd)

/cluster/home/skyriakos/chemprop_run/git/notebooks


In [5]:
os.chdir('..')
import deepadr
from deepadr.dataset import *
from deepadr.utilities import *
from deepadr.run_workflow import *
from deepadr.chemfeatures import *
from deepadr.hyphelper import *
# from deepadr.model_gnn import GCN as testGCN
from deepadr.model_gnn_ogb import GNN, DeepAdr_SiameseTrf, ExpressionNN
# from deepadr.model_attn_siamese import *
from ogb.graphproppred import Evaluator
os.chdir(cwd)

In [6]:
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw

In [7]:
# from tdc.single_pred import Tox
# from tdc.multi_pred import DDI

In [8]:
rawdata_dir = '../data/raw/'
processed_dir = '../data/processed/'
up_dir = '..'

In [9]:
report_available_cuda_devices()

number of GPUs available: 8
cuda:0, name:GeForce GTX 1080 Ti
total memory available: 10.91650390625 GB
total memory allocated on device: 0.0 GB
max memory allocated on device: 0.0 GB
total memory cached on device: 0.0 GB
max memory cached  on device: 0.0 GB

cuda:1, name:GeForce GTX 1080 Ti
total memory available: 10.91650390625 GB
total memory allocated on device: 0.0 GB
max memory allocated on device: 0.0 GB
total memory cached on device: 0.0 GB
max memory cached  on device: 0.0 GB

cuda:2, name:GeForce GTX 1080 Ti
total memory available: 10.91650390625 GB
total memory allocated on device: 0.0 GB
max memory allocated on device: 0.0 GB
total memory cached on device: 0.0 GB
max memory cached  on device: 0.0 GB

cuda:3, name:GeForce GTX 1080 Ti
total memory available: 10.91650390625 GB
total memory allocated on device: 0.0 GB
max memory allocated on device: 0.0 GB
total memory cached on device: 0.0 GB
max memory cached  on device: 0.0 GB

cuda:4, name:GeForce GTX 1080 Ti
total memory av

In [10]:
n_gpu = torch.cuda.device_count()
n_gpu

8

In [11]:
device_cpu = get_device(to_gpu=False)
# device_gpu = get_device(True, index=0)

# fdtype = torch.float32

In [12]:
print("torch:", torch.__version__)
print("CUDA:", torch.version.cuda)
print(sys.version)

torch: 1.9.1
CUDA: 11.1
3.9.9 | packaged by conda-forge | (main, Dec 20 2021, 02:41:03) 
[GCC 9.4.0]


In [13]:
# print(torch.cuda.memory_summary(device=device_gpu))

### Preparing dataset 

In [14]:
score = 'total_thresh'
score_val = 3

In [15]:
# v_1: GNN
# v_2: Alt Models

In [16]:
# TDC Tox
# DSdataset_name = 'SynergxZloewe' #'OncoPolyPharmacology' #'DrugComb'
DSdataset_name = f'DrugComb_{score}_{score_val}' #'DrugComb'


#fname_suffix = ds_config["fname_suffix"]
# similarity_types = ['chem']
# kernel_option = 'sqeuclidean'
data_fname = 'data_v2'
# interact_matfname = ds_config["interact_matfname"]
# exp_iden = 'simtypeall'
# ddi_interaction_labels_pth = ds_config["ddi_interaction_labels_pth"]

# up_dir, processed_dir, DSdataset_name, data_fname

In [17]:
targetdata_dir = create_directory(os.path.join(processed_dir, DSdataset_name, data_fname))
targetdata_dir_raw = create_directory(os.path.join(targetdata_dir, "raw"))
targetdata_dir_processed = create_directory(os.path.join(targetdata_dir, "processed"))
targetdata_dir_exp = create_directory(os.path.join(targetdata_dir, "experiments"))
# # ReaderWriter.dump_data(dpartitions, os.path.join(targetdata_dir, 'data_partitions.pkl'))
print(targetdata_dir)

path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
/cluster/home/skyriakos/chemprop_run/git/data/processed/DrugComb_total_thresh_3/data_v2


In [18]:
xFlat = ReaderWriter.read_data(os.path.join(targetdata_dir_raw, 'X_flat.pkl'))
y = ReaderWriter.read_data(os.path.join(targetdata_dir_raw, 'y.pkl'))
expression = ReaderWriter.read_data(os.path.join(targetdata_dir_raw, 'expression.pkl'))

In [19]:
xFlatMat = torch.stack([torch.cat(i) for i in list(xFlat.values())])
xFlatMat.shape

torch.Size([64140, 18])

In [20]:
torch.tensor(expression)

tensor([[ 0.1790,  0.2520,  1.3661,  ..., -0.4165,  0.9541,  0.9106],
        [ 0.1790,  0.2520,  1.3661,  ..., -0.4165,  0.9541,  0.9106],
        [ 0.1790,  0.2520,  1.3661,  ..., -0.4165,  0.9541,  0.9106],
        ...,
        [-0.0309, -1.4168,  1.5472,  ..., -1.2610,  1.2777,  0.9653],
        [-0.0309, -1.4168,  1.5472,  ..., -1.2610,  1.2777,  0.9653],
        [-0.0309, -1.4168,  1.5472,  ..., -1.2610,  1.2777,  0.9653]],
       dtype=torch.float64)

In [21]:
x = torch.cat([xFlatMat, torch.tensor(expression)], dim=1)
x.shape

torch.Size([64140, 926])

In [22]:
# %%time

# # Make sure to first run the "data_generation" notebook first

# dataset = MoleculeDataset(root=targetdata_dir)

In [23]:
# print()
# print(f'Dataset: {dataset}:')
# print('====================')
# print(f'Number of graphs: {len(dataset)}')
# print(f'Number of features: {dataset.num_features}')
# print(f'Number of classes: {dataset.num_classes}')

# # data0 = dataset[0]  # Get the first graph object.

# # print()
# # print(data)
# # print('=============================================================')

# # # Gather some statistics about the first graph.
# # print(f'Number of nodes: {data.num_nodes}')
# # print(f'Number of edges: {data.num_edges}')
# # print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
# # print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
# # print(f'Contains self-loops: {data.contains_self_loops()}')
# # print(f'Is undirected: {data.is_undirected()}')

In [24]:
# data0

In [25]:
# data0.expression.shape

In [26]:
# data0.y

In [27]:
# dataset.tensors[1]

In [28]:
# used_dataset = dataset

# If you want to use a smaller subset of the dataset for testing
# smaller_dataset_len = int(len(dataset)/1)
# used_dataset = dataset[:smaller_dataset_len]

In [29]:
# type(used_dataset)

In [30]:
# dataset.data.y

In [31]:
y

array([0, 0, 0, ..., 1, 0, 0])

In [32]:
# fold_partitions = get_stratified_partitions(dataset.data.y,
#                                             num_folds=5,
#                                             valid_set_portion=0.1,
#                                             random_state=42)

In [33]:
fold_partitions = get_stratified_partitions(y,
                                            num_folds=5,
                                            valid_set_portion=0.1,
                                            random_state=42)

fold_num: 0
train data
class: 0 norm count: 0.6346470333477696
class: 1 norm count: 0.3653529666522304
validation data
class: 0 norm count: 0.6346453624318005
class: 1 norm count: 0.3653546375681995
test data
class: 0 norm count: 0.6347053320860617
class: 1 norm count: 0.3652946679139383

-------------------------
fold_num: 1
train data
class: 0 norm count: 0.6346470333477696
class: 1 norm count: 0.3653529666522304
validation data
class: 0 norm count: 0.6346453624318005
class: 1 norm count: 0.3653546375681995
test data
class: 0 norm count: 0.6347053320860617
class: 1 norm count: 0.3652946679139383

-------------------------
fold_num: 2
train data
class: 0 norm count: 0.634668687743612
class: 1 norm count: 0.36533131225638804
validation data
class: 0 norm count: 0.6346453624318005
class: 1 norm count: 0.3653546375681995
test data
class: 0 norm count: 0.6346273776114749
class: 1 norm count: 0.3653726223885251

-------------------------
fold_num: 3
train data
class: 0 norm count: 0.634668

In [34]:
# train_val_test_frac = [0.7, 0.1, 0.2]
# assert sum(train_val_test_frac) == 1

# torch.manual_seed(42)
# used_dataset = used_dataset.shuffle()

# num_train = round(train_val_test_frac[0] * len(used_dataset)) 
# num_trainval = round((train_val_test_frac[0]+train_val_test_frac[1]) * len(used_dataset))

# train_dataset = used_dataset[:num_train]
# val_dataset = used_dataset[num_train:num_trainval]
# test_dataset = used_dataset[num_trainval:]

In [35]:
# train_dataset = Subset(used_dataset, fold_partitions[0]['train'])
# val_dataset = Subset(used_dataset, fold_partitions[0]['validation'])
# test_dataset = Subset(used_dataset, fold_partitions[0]['test'])

In [36]:
# print(f'Number of training graphs: {len(train_dataset)}')
# print(f'Number of val graphs: {len(val_dataset)}')
# print(f'Number of test graphs: {len(test_dataset)}')

In [37]:
print("Number of training graphs: "+ str(len(fold_partitions[0]['train'])))
print("Number of validation graphs: "+ str(len(fold_partitions[0]['validation'])))
print("Number of testing graphs: "+ str(len(fold_partitions[0]['test'])))


Number of training graphs: 46180
Number of validation graphs: 5132
Number of testing graphs: 12828


### Alt Models

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

In [39]:
x_np = x.numpy()
y_np = y

In [40]:
scaler = StandardScaler()
x_np_scale = scaler.fit_transform(x_np)

In [41]:
# part_train_x = np.take(x_np_scale, fold_partitions[0]['train'], axis=0)
# part_train_y = np.take(y_np, fold_partitions[0]['train'], axis=0)
# part_train_x.shape

In [42]:
# part_test_x = np.take(x_np_scale, fold_partitions[0]['test'], axis=0)
# part_test_y = np.take(y_np, fold_partitions[0]['test'], axis=0)
# part_test_x.shape

In [43]:
# %%time

# clf = LogisticRegression(max_iter=10000, random_state=42, solver="sag").fit(part_train_x, part_train_y)

In [44]:
targetdata_dir_exp

'/cluster/home/skyriakos/chemprop_run/git/data/processed/DrugComb_total_thresh_3/data_v2/experiments'

In [45]:
model = LogisticRegression(max_iter=10000,
                                      random_state=42,
                                      solver="lbfgs",
                                      penalty='l2',
                                      l1_ratio=None,
                                      verbose=1)

In [46]:
# model = DecisionTreeClassifier()

In [47]:
time_stamp = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
print("Start: " + time_stamp)

for q_i in range(len(fold_partitions)):
# for q_i in range(1):

    partition = fold_partitions[q_i]
    
    ids_train = partition['train']
    ids_test = partition['test']
    
    part_train_x = np.take(x_np_scale, ids_train, axis=0)
    part_train_y = np.take(y_np, ids_train, axis=0)
    
    part_test_x = np.take(x_np_scale, ids_test, axis=0)
    part_test_y = np.take(y_np, ids_test, axis=0)
    
    exp_dir = create_directory(os.path.join(targetdata_dir_exp, "fold_"+str(q_i)+"_"+time_stamp))
    create_directory(os.path.join(exp_dir, "predictions"))
    
    model_fit = model.fit(part_train_x, part_train_y)
    
    prob_scores_arr = model_fit.predict_proba(part_test_x)
    pred_class = model_fit.predict(part_test_x)
    ref_class = part_test_y
    epoch = 0
    dsettype = 'test'

    dset_perf = perfmetric_report(pred_class, ref_class, prob_scores_arr[:,1], epoch,
                              outlog = os.path.join(exp_dir, dsettype + ".log"))
    
    predictions_df = build_predictions_df(ids_test, ref_class, pred_class, prob_scores_arr)
    predictions_df.to_csv(os.path.join(exp_dir, 'predictions', f'epoch_{epoch}_predictions_{dsettype}.csv'))
    
print("End: " + datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))

Start: 2022-03-24_16-02-38
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          927     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.20095D+04    |proj g|=  6.21800D+03


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
 This problem is unconstrained.



At iterate   50    f=  2.73016D+04    |proj g|=  8.28918D+01

At iterate  100    f=  2.72535D+04    |proj g|=  2.50565D+01

At iterate  150    f=  2.72456D+04    |proj g|=  1.76240D+01

At iterate  200    f=  2.72443D+04    |proj g|=  6.11669D+00

At iterate  250    f=  2.72437D+04    |proj g|=  4.42540D+00

At iterate  300    f=  2.72433D+04    |proj g|=  4.34195D+00

At iterate  350    f=  2.72432D+04    |proj g|=  2.23140D+00

At iterate  400    f=  2.72432D+04    |proj g|=  1.14586D+00

At iterate  450    f=  2.72431D+04    |proj g|=  8.71583D-01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
  927    473    511   

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   32.3s finished


path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          927     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.20095D+04    |proj g|=  6.21800D+03


...

path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
path_current_dir /cluster/home/skyriakos/chemprop_run/git/deepadr
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =          927     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  3.20095D+04    |proj g|=  6.21900D+03


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
 This problem is unconstrained.



At iterate   50    f=  2.71983D+04    |proj g|=  7.07238D+01

At iterate  100    f=  2.71511D+04    |proj g|=  2.45174D+01

At iterate  150    f=  2.71439D+04    |proj g|=  1.77910D+01

At iterate  200    f=  2.71426D+04    |proj g|=  7.11643D+00

At iterate  250    f=  2.71422D+04    |proj g|=  3.90215D+00

At iterate  300    f=  2.71419D+04    |proj g|=  4.48465D+00

At iterate  350    f=  2.71417D+04    |proj g|=  5.01694D+00

At iterate  400    f=  2.71416D+04    |proj g|=  4.02970D+00

At iterate  450    f=  2.71416D+04    |proj g|=  8.77997D-01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
  927    490    520   

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   31.8s finished


End: 2022-03-24_16-05-39


### Scores

In [48]:
# time_stamp = "2022-03-10_18-42-45"

import glob
exp_dirs = glob.glob(targetdata_dir_exp+"/fold_*_"+time_stamp)
len(exp_dirs)

5

In [49]:
folds = []

for edir in exp_dirs:
    fold = edir.split("/")[-1].split('_')[1]
    with open(os.path.join(edir, "test.log")) as f:
        lines = f.read().splitlines()
#         print(lines[17:23])
        folds.append([fold, float(lines[18]), float(lines[22])])

In [50]:
df_folds = pd.DataFrame(data=folds, columns=["Fold","AUPR", "AUC"]).set_index("Fold")
# df_folds["Fscore"] = df_folds.apply(lambda x: (2*x[0]*x[1])/(x[0]+x[1]), axis=1) # harmonic mean of AUC, AUPR
df_folds

,AUPR,AUC
Fold,,
0,0.599707,0.701583
1,0.594751,0.696580
2,0.588012,0.697524
3,0.593316,0.699101
4,0.590234,0.692597


In [51]:
df_folds.mean(axis=0).values

array([0.5932038 , 0.69747682])

In [52]:
F_score(*df_folds.mean(axis=0).values)

0.6411282414758441